In [11]:
import os
import h5py
import os.path as osp

# Path to the HDF5 file
out_file_dino = osp.join("/media/ekoller/T7/Features2D/dino_segmentation", "DinoV2", "patch_32_18", "avg", "fcf66d9e-622d-291c-84c2-bb23dfe31327.h5")
out_file_proj = osp.join("/media/ekoller/T7/Features2D/projection", "DinoV2", "patch_32_18", "median", "02b33dfb-be2b-2d54-92d2-cd012b2b3c40.h5")
out_file_segmentation = osp.join("/media/ekoller/T7/Segmentation", "DinoV2", "objects", "0cac7652-8d6f-2d13-8f0e-a1dd07a8a847.h5")
# Initialize an empty dictionary to store the features
features = {}

# # Open the HDF5 file for reading
# with h5py.File(out_file_proj, 'r') as hdf_file:
#     # Iterate over each object ID (which corresponds to the dataset keys)
#     for object_key in hdf_file.keys():
#         # Read the dataset corresponding to the object_key
#         stacked_features = hdf_file[object_key][:]
        
#         # Convert the string key back to the original object_id if necessary
#         object_id = object_key
        
#         # Store the feature list in the dictionary
#         features[object_id] = [stacked_features[i] for i in range(stacked_features.shape[0])]

# # Now, you can access features[obj_id] as a list of feature vectors
# # Print the contents to verify everything is correct

# print("stuff for the projection file")
# for object_id, feature_list in features.items():
#     print(f"Object ID: {object_id}")
#     for i, feature_vector in enumerate(feature_list):
#         print(f"  Feature vector {i}: {feature_vector}")






# features_proj = {}

# with h5py.File(out_file_dino, 'r') as hdf_file:
#     # Iterate over each frame_idx (which corresponds to the groups in the HDF5 file)
#     for frame_idx in hdf_file.keys():
#         # Initialize a dictionary for each frame_idx
#         features_proj[frame_idx] = {}
        
#         # Access the group corresponding to the current frame_idx
#         frame_group = hdf_file[frame_idx]
        
#         # Iterate over each object_id within the current frame_idx group
#         for object_key in frame_group.keys():
#             # Convert object_key back to object_id if necessary
#             object_id = object_key
        
#             # Retrieve the feature vector from the dataset
#             feature_vector = frame_group[object_key][:]
            
#             # Store the feature vector in the dictionary under the object_id
#             features_proj[frame_idx][object_id] = feature_vector

# # Now, you can access features[frame_idx][object_id] to get the feature vector
# # Print the contents to verify everything is correct
# print("stuff for the dino segmentation")
# for frame_idx, object_dict in features_proj.items():
#     print(f"Frame Index: {frame_idx}")
#     for object_id, feature_vector in object_dict.items():
#         print(f"  Object ID: {object_id}")
#         print(f"    Feature Vector: {feature_vector.shape}")


#for the segmentation thing
# Initialize the features dictionary
features_seg = {}
# Open the HDF5 file in read mode ('r')
with h5py.File(out_file_segmentation, 'r') as hdf_file:
    for frame_idx in hdf_file.keys():
        # Initialize boxlist for current frame
        bounding_boxes = []
        
        # Get info
        frame_group = hdf_file[frame_idx]
        
        # Iterate over each bounding box
        for bbox_key in frame_group.keys():
            bbox_group = frame_group[bbox_key]
            
            # Get the object ID
            object_id = bbox_group.attrs['object_id']
            
            # Get the bounding box
            bbox = bbox_group['bbox'][:]
            
            # Get the mask
            mask = bbox_group['mask'][:]
            
            # Append to list
            bounding_boxes.append({
                'object_id': object_id,
                'bbox': bbox,
                'mask': mask
            })
        
        # Store it to the corresponding frame
        features_seg[frame_idx] = bounding_boxes

# # Print features to debug
# for frame_idx, boxes in features_seg.items():
#     print(f"Frame: {frame_idx}")
#     for box in boxes:
#         print(f"  Object ID: {box['object_id']}")
#         print(f"  BBox: {box['bbox']}")
#         print(f"  Mask Shape: {box['mask'].shape}")


print(features_seg["000003"])







[{'object_id': 1, 'bbox': array([300, 180,  90,  90], dtype=int32), 'mask': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}, {'object_id': 2, 'bbox': array([150, 120, 150, 240], dtype=int32), 'mask': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}, {'object_id': 11, 'bbox': array([600,  30, 360, 510], dtype=int32), 'mask': array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 225, 225, 225],
       [  0,   0,   0, ..., 225, 225, 225],
       [  0,   0,   0, ..., 225, 225, 225]], dtype=uint8)}, {'object_id': 3, 'bbox': array([300, 300,

In [1]:
#look at the mask data
import pickle
import os.path as osp


data_dir ='/local/home/ekoller/R3Scan'
data_path=  osp.join(data_dir, "Results","mask_metric/mask_metric.pkl" )


with open(data_path, 'rb') as file:
    data = pickle.load(file)  # This loads the dictionary from the pickle file

# Assuming 'data' is a dictionary where keys are scan IDs and values are numbers
# Sort by the values in descending order
sorted_data = sorted(data.items(), key=lambda x: x[1], reverse=True)

# Print the sorted scan IDs and their corresponding numbers
for scan_id, number in sorted_data:
    print(f"Scan ID: {scan_id}, Number: {number}")




Scan ID: 6bde609b-9162-246f-8f90-c3d2444a5ab8, Number: 0.6793296354660793
Scan ID: 1d234010-e280-2b1a-8da8-205855a16b6b, Number: 0.6154761882555563
Scan ID: 0cac7536-8d6f-2d13-8dc2-2f9d7aa62dc4, Number: 0.6053620138288301
Scan ID: 422885bd-192d-25fc-8571-abff2237f383, Number: 0.5932973399042338
Scan ID: 1d23401e-e280-2b1a-8cef-f28df9df2618, Number: 0.5785955329259019
Scan ID: 501ebf0b-a3bb-263f-86fd-7ef000a19588, Number: 0.5646181646214503
Scan ID: 55551077-36f1-29c0-89ec-2e7690991cb2, Number: 0.5642519928329063
Scan ID: 54b263a3-0199-2df4-87db-40539528902d, Number: 0.549442684382646
Scan ID: 0ad2d39b-79e2-2212-99ae-830c292cd079, Number: 0.5453834049148543
Scan ID: 6bde608d-9162-246f-8dde-3f158d134d50, Number: 0.5421325532215256
Scan ID: 6bde6079-9162-246f-8fad-d8f2358bd413, Number: 0.5383594750765853
Scan ID: 47319774-f9f7-2a1a-9412-d4a1c89c8aa3, Number: 0.5344196064678896
Scan ID: 1d234018-e280-2b1a-8f22-da2b997865e8, Number: 0.529309130565892
Scan ID: 6bde60c0-9162-246f-8d1f-32543ba